# Document Intelligence Pipeline for Legal Analytics

## Overview
This notebook demonstrates an automated document ingestion and analysis pipeline for a legal firm. The pipeline:
1. **Ingests** PDF documents (invoices, contracts, emails, meeting minutes)
2. **Classifies** documents by type using Qwen LLM
3. **Extracts** structured fields (client names, amounts, dates, parties)
4. **Validates** and stores data for downstream analytics

## Technical Stack
- **PDF Extraction**: pdfplumber
- **LLM**: Qwen 2.5 (7B) via Ollama
- **Validation**: Pydantic
- **Data Processing**: Pandas

---

## Setup and Imports

In [ ]:
import sys
sys.path.append('..')

from src.ingestion import DocumentIngestor
from src.classifier import DocumentClassifier
from src.extractor import FieldExtractor
from src.schemas import DocumentType, create_document
from src.utils import (
    save_to_json, 
    save_to_csv, 
    create_summary_report,
    display_document_table,
    validate_ollama_connection
)

import pandas as pd
import json
from pathlib import Path

# Configure display options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 50)

print("✓ All modules imported successfully")

## Step 1: Verify Ollama Connection

In [ ]:
# Check if Ollama is running and qwen2.5:7b is available
is_connected = validate_ollama_connection(model_name="qwen2.5:7b")

if not is_connected:
    print("\n⚠️  Please ensure Ollama is running: ollama serve")
    print("⚠️  And qwen2.5:7b is installed: ollama pull qwen2.5:7b")

## Step 2: Document Ingestion

Extract text and metadata from PDF documents using pdfplumber.

In [ ]:
# Initialize the document ingestor
ingestor = DocumentIngestor()

# Ingest all PDFs from the input directory
input_dir = "../data/input"
documents = ingestor.batch_ingest(input_dir)

print(f"\n📄 Successfully ingested {len(documents)} documents\n")

# Display summary of ingested documents
for i, doc in enumerate(documents, 1):
    metadata = doc['metadata']
    print(f"{i}. {metadata['file_name']}")
    print(f"   Pages: {metadata['num_pages']}")
    print(f"   Size: {metadata['file_size'] / 1024:.1f} KB")
    print(f"   Has Tables: {metadata['has_tables']}")
    print(f"   Text Length: {len(doc['text'])} characters\n")

### Preview First Document

In [ ]:
if documents:
    print("First 500 characters of the first document:\n")
    print(documents[0]['text'][:500])
    print("\n...\n")

## Step 3: Document Classification

Use Qwen LLM to classify each document into one of four types:
- Invoice
- Contract
- Email
- Meeting Minutes

In [ ]:
# Initialize classifier
classifier = DocumentClassifier(model_name="qwen2.5:7b")

# Classify each document
print("🤖 Classifying documents...\n")
classifications = []

for i, doc in enumerate(documents, 1):
    print(f"Classifying document {i}/{len(documents)}: {doc['metadata']['file_name']}")
    doc_type, confidence = classifier.classify(doc['text'])
    
    classifications.append({
        'document': doc,
        'type': doc_type,
        'confidence': confidence
    })
    
    print(f"   → Type: {doc_type.upper()} (confidence: {confidence:.2%})\n")

print("✓ Classification complete")

### Classification Results Summary

In [ ]:
# Create summary DataFrame
classification_summary = pd.DataFrame([
    {
        'File': c['document']['metadata']['file_name'],
        'Document Type': c['type'].upper(),
        'Confidence': f"{c['confidence']:.1%}"
    }
    for c in classifications
])

display(classification_summary)

## Step 4: Field Extraction

Extract structured fields from each document based on its type:
- **Invoice**: client name, amount, date, vendor, etc.
- **Contract**: parties, value, dates, terms
- **Email**: sender, recipients, subject, date
- **Meeting Minutes**: attendees, agenda, decisions, action items

In [ ]:
# Initialize extractor
extractor = FieldExtractor(model_name="qwen2.5:7b")

# Extract fields from each document
print("🔍 Extracting structured fields...\n")
extracted_documents = []

for i, item in enumerate(classifications, 1):
    doc = item['document']
    doc_type = item['type']
    confidence = item['confidence']
    
    print(f"Extracting from document {i}/{len(classifications)}: {doc['metadata']['file_name']}")
    
    # Extract fields
    extracted_fields = extractor.extract(doc['text'], doc_type)
    
    # Create document object using Pydantic schema
    try:
        document_obj = create_document(
            doc_type=DocumentType(doc_type),
            file_name=doc['metadata']['file_name'],
            confidence_score=confidence,
            **extracted_fields
        )
        extracted_documents.append(document_obj)
        print(f"   ✓ Extracted {len(extracted_fields)} fields\n")
    except Exception as e:
        print(f"   ⚠️  Error creating document object: {str(e)}\n")

print(f"✓ Extraction complete: {len(extracted_documents)} documents processed")

### View Extracted Data

In [ ]:
# Display extracted data for each document
for i, doc in enumerate(extracted_documents, 1):
    print(f"\n{'='*80}")
    print(f"Document {i}: {doc.file_name}")
    print(f"Type: {doc.document_type.upper()}")
    print(f"Confidence: {doc.confidence_score:.1%}")
    print(f"{'='*80}\n")
    
    # Display in pretty JSON format (excluding raw_text)
    doc_dict = doc.dict()
    doc_dict.pop('raw_text', None)  # Remove raw text for cleaner display
    print(json.dumps(doc_dict, indent=2, default=str))
    print()

## Step 5: Data Validation & Storage

Validate extracted data using Pydantic schemas and save to:
- Individual JSON files (for document retrieval)
- Master CSV file (for analytics)

In [ ]:
# Save to JSON (individual files)
print("💾 Saving documents...\n")
save_to_json(extracted_documents, output_dir="../data/output/json")

# Save to CSV (master file)
df = save_to_csv(extracted_documents, output_file="../data/output/master_data.csv")

print("\n✓ All documents saved successfully")

### Preview Master CSV

In [ ]:
# Display first few rows of the CSV
print("Master Data CSV Preview:\n")
display(df.head())

## Step 6: Summary & Analytics

Demonstrate downstream applications:
- Summary statistics
- Search capabilities
- Aggregation examples

In [ ]:
# Create summary report
summary = create_summary_report(extracted_documents)

print("📊 PROCESSING SUMMARY\n")
print(f"Total Documents Processed: {summary['total_documents']}")
print(f"Average Confidence Score: {summary['average_confidence']:.1%}")
print(f"\nDocuments by Type:")
for doc_type, count in summary['documents_by_type'].items():
    print(f"  - {doc_type.upper()}: {count}")

if summary['total_invoice_amount'] > 0:
    print(f"\nFinancial Summary:")
    print(f"  Total Invoice Amount: ${summary['total_invoice_amount']:.2f}")
    print(f"  Number of Invoices: {summary['num_invoices_with_amounts']}")

### Document Summary Table

In [ ]:
# Create a clean summary table
summary_table = display_document_table(extracted_documents)
display(summary_table)

## Step 7: Downstream Use Cases

### A. Search Example

In [ ]:
# Example: Find all invoices over a certain amount
print("🔍 Search Example: Find invoices over $50\n")

high_value_invoices = [
    doc for doc in extracted_documents 
    if doc.document_type == 'invoice' and doc.total_amount and doc.total_amount > 50
]

if high_value_invoices:
    for inv in high_value_invoices:
        print(f"- {inv.file_name}: ${inv.total_amount:.2f} from {inv.vendor_name}")
else:
    print("No invoices found over $50")

### B. Aggregation Example

In [ ]:
# Example: Total spending by vendor
print("📈 Aggregation Example: Total spending by vendor\n")

vendor_totals = {}
for doc in extracted_documents:
    if doc.document_type == 'invoice' and doc.vendor_name and doc.total_amount:
        vendor = doc.vendor_name
        vendor_totals[vendor] = vendor_totals.get(vendor, 0) + doc.total_amount

if vendor_totals:
    for vendor, total in sorted(vendor_totals.items(), key=lambda x: x[1], reverse=True):
        print(f"- {vendor}: ${total:.2f}")
else:
    print("No vendor data available")

### C. Date-based Query Example

In [ ]:
# Example: Find all documents from a specific date range
print("📅 Query Example: Documents from June 2025\n")

june_docs = []
for doc in extracted_documents:
    date_field = None
    if doc.document_type == 'invoice' and doc.invoice_date:
        date_field = doc.invoice_date
    elif doc.document_type == 'email' and doc.email_date:
        date_field = doc.email_date
    
    if date_field and '2025-06' in str(date_field):
        june_docs.append((doc.file_name, doc.document_type, date_field))

if june_docs:
    for filename, doc_type, date in june_docs:
        print(f"- {filename} ({doc_type}) - {date}")
else:
    print("No documents found from June 2025")

### D. Export for Reporting

In [ ]:
# Create a DataFrame for easy export to Excel/PowerBI
print("📤 Creating export-ready DataFrame\n")

# Invoice-specific export
invoice_data = []
for doc in extracted_documents:
    if doc.document_type == 'invoice':
        invoice_data.append({
            'Invoice Number': doc.invoice_number,
            'Date': doc.invoice_date,
            'Client': doc.client_name,
            'Vendor': doc.vendor_name,
            'Amount': doc.total_amount,
            'Currency': doc.currency,
            'Confidence': f"{doc.confidence_score:.1%}"
        })

if invoice_data:
    invoice_df = pd.DataFrame(invoice_data)
    print("Invoice Summary for Reporting:\n")
    display(invoice_df)
    
    # Save to CSV for Excel
    invoice_df.to_csv('../data/output/invoice_report.csv', index=False)
    print("\n✓ Invoice report saved to data/output/invoice_report.csv")
else:
    print("No invoice data to export")

## Summary

### Pipeline Capabilities Demonstrated:

✅ **Ingestion**: Successfully extracted text from PDF documents

✅ **Classification**: Accurately classified documents by type using LLM

✅ **Extraction**: Extracted all required fields:
   - Client Name
   - Invoice Amount
   - Date(s)
   - Involved Parties

✅ **Validation**: Used Pydantic schemas for data validation

✅ **Storage**: Saved in multiple formats (JSON, CSV) for different use cases

✅ **Analytics**: Demonstrated search, aggregation, and reporting capabilities

### Next Steps for Production:
- Add batch processing with parallel execution
- Implement proper error handling and retry logic
- Add vector database for semantic search
- Create REST API endpoints
- Add monitoring and logging
- Implement data warehouse integration